# Brief Summary of GDOT Phase I 1D Thermal Modeling


<!-- #### _Basic thermal model_ -->

In modeling the thermal behavior of curing concrete, a function $\dot e_{gen}$ describing the energy release (heat of hydration) is used in the heat diffusion equation:

$$ \rho c_v { {\partial T} \over {\partial t}} = k { \nabla^2 T} + \dot e_{gen} $$

In one dimension:

$$ \rho c_v { {\partial T} \over {\partial t}} = k { {\partial^2 T} \over {\partial z^2}} + \dot e_{gen} $$

$\rho$, $c_v$, and $k$ are the density, specific heat, and thermal conductivity, respectively, of the concrete.  $T$ is concrete temperature, $t$ is time, and $z$ is a spatial coordinate.

An often-used form of $\dot e_{gen}$ is based on the Arrhenius equation and equivalent age:

$$ \dot e_{gen} = H_u \cdot C_c \cdot \alpha(t_e) \cdot \bigg( { {\tau} \over {t_e} } \bigg )^\beta \bigg( { {\beta} \over {t_e} } \bigg ) \cdot exp \bigg( {{E_a} \over {R} }\bigg[ {{1} \over {T_r}} - {{1} \over {T}}  \bigg ]  \bigg ) $$

The first two parameters are:

- $H_u$ : total heat of hydration of cementious material, J/g
- $C_c$ : mass of cementitious material to volume of concrete, g/m^3

$\alpha(t_e)$ is the degree of hydration 

$$ \alpha(t_e) = \alpha_u \cdot exp \bigg( - \bigg[ {{\tau} \over {t_e}} \bigg]^{\beta} \bigg) $$ 

where $\alpha_u$ is the (dimensionless) degree of ultimate hydration, $0 \leq \alpha_u \leq 1$.  $\beta$ is a dimensionless shape parameter, and $\tau$ is a time parameter in hours or seconds.  Together, $\alpha_u$, $\beta$, and $\tau$ control the shape of the exponential curves.

$t_e$ is the equivalent age, a nonlinearly scaled cumulative time:

$$ t_e = \int_{t=0}^{t} exp \bigg(- {{E_a} \over {R} }\bigg[ {{1} \over {T_r}} - {{1} \over {T}}  \bigg ]  \bigg ) dt $$

in discrete terms:

$$ t_e = \sum_{t=0}^{t} exp \bigg(- {{E_a} \over {R} }\bigg[ {{1} \over {T_r}} - {{1} \over {T}}  \bigg ]  \bigg ) \Delta t $$

where

- $E_a$ : activation energy, J/mol
- $R$   : the universal gas constant (J/mol/K)
- $T_r$ : a reference temperature, K

Empirically derived regression equations for  $H_u$, $C_c$,  $\alpha_u$, $\beta$, $\tau$, and $ E_a $ are given in, e.g.:

- Schindler, Anton K., Terry Dossey, and B.F. McCullough. 2002. “Temperature Control During Construction to Improve the Long Term Performance of Portland Cement Concrete Pavements.” Vol. 7.
- Schindler, Anton K., and Kevin J. Folliard. 2005. “Heat of Hydration Models for Cementitious Materials.” ACI Materials Journal 102 (1): 24–33. doi:10.1680/adcr.11.00007.
- Riding, Kyle A., Jonathan L. Poole, Kevin J. Folliard, Maria C G Juenger, and Anton K. Schindler. 2012. “Modeling Hydration of Cementitious Systems.” ACI Materials Journal 109 (2): 225–34.

This $\dot e_{gen}$ model will be referred to as the _Arhennius/Schindler_ or _Schindler_ model.

#### _Simulating the Arrhenius/Schindler model in adiabatic conditions_

A simple Ptyhon code has been used to simulate the time-dependent adiabatic behavior of several different concrete mixes under adiabatic conditions and various initial temperatures.  Under adiabatic conditions the dependence of spatial coordinates vanishes, i.e. $\nabla^2 T = 0$, and therefore boundary conditions are irrelevant.  The thermal model is thus

$$ \rho c_v { {\partial T} \over {\partial t}} = \dot e_{gen} $$

with the initial condition being $T(t=0) = T_i$.

Below some of the $\dot e_{gen}$ curves are plotted for the AA baseline mix and 45% fly ash mixes from Phase I of the GDOT mass concrete project.  For each mix, there are three initial temperatures shown:

- 10$^\circ$C
- 20$^\circ$C
- 30$^\circ$C

Note that $\dot e_{gen}$ depends strongly on concrete temperature.

In [1]:
# some imports
import numpy as np
import pandas as pd
from bokeh.io import output_notebook, show
output_notebook()
from bokeh.plotting import figure

Loading BokehJS ...

In [2]:
# read in data for AA baseline and 45% flyash for initial temperatures of 10C, 20C, and 30C
AA10C = pd.read_excel('CCTadiabatic_Output_AA+_Baseline_Tinit_10C.xlsx', 'AdiabaticConds', index_col=None, usecols=['t_h', 'egenadbtc_W*m-3'], na_values=['NA'])
AA20C = pd.read_excel('CCTadiabatic_Output_AA+_Baseline_Tinit_20C.xlsx', 'AdiabaticConds', index_col=None, usecols=['t_h', 'egenadbtc_W*m-3'], na_values=['NA'])
AA30C = pd.read_excel('CCTadiabatic_Output_AA+_Baseline_Tinit_30C.xlsx', 'AdiabaticConds', index_col=None, usecols=['t_h', 'egenadbtc_W*m-3'], na_values=['NA'])

FourFiveFA10C = pd.read_excel('CCTadiabatic_Output_AA+_45percent_FlyAsh_Tinit_10C.xlsx', 'AdiabaticConds', index_col=None, usecols=['t_h', 'egenadbtc_W*m-3'], na_values=['NA'])
FourFiveFA20C = pd.read_excel('CCTadiabatic_Output_AA+_45percent_FlyAsh_Tinit_20C.xlsx', 'AdiabaticConds', index_col=None, usecols=['t_h', 'egenadbtc_W*m-3'], na_values=['NA'])
FourFiveFA30C = pd.read_excel('CCTadiabatic_Output_AA+_45percent_FlyAsh_Tinit_30C.xlsx', 'AdiabaticConds', index_col=None, usecols=['t_h', 'egenadbtc_W*m-3'], na_values=['NA'])

# tools for Bokeh figures
TOOLS = 'pan,box_zoom,wheel_zoom,box_select,reset,hover,crosshair,save'

# plot data
datFig01 = figure(#plot_width=1200, plot_height=700,
                  plot_width=900, plot_height=600,
                  tools=TOOLS,
                  title="Simulated Adiabatic $\dot e_{gen}$",
                  x_axis_label='Time, hours',
                  x_range=[0,72],
                  y_axis_label='W/m^3',
                  y_range=[0,5200]
)

datFig01.line(AA10C['t_h'], AA10C['egenadbtc_W*m-3'], line_width=2, line_color='#DE6C6F', legend="AA Baseline, T_i = 10C")
datFig01.line(AA20C['t_h'], AA20C['egenadbtc_W*m-3'], line_width=2, line_color='#A8383B', legend="AA Baseline, T_i = 20C")
datFig01.line(AA30C['t_h'], AA30C['egenadbtc_W*m-3'], line_width=2, line_color='#731517', legend="AA Baseline, T_i = 30C")

datFig01.line(FourFiveFA10C['t_h'], FourFiveFA10C['egenadbtc_W*m-3'], line_dash='dashed', line_width=2, line_color='#DE6C6F', legend="45% Fly Ash, T_i = 10C")
datFig01.line(FourFiveFA20C['t_h'], FourFiveFA20C['egenadbtc_W*m-3'], line_dash='dashed', line_width=2, line_color='#A8383B', legend="45% Fly Ash, T_i = 20C")
datFig01.line(FourFiveFA30C['t_h'], FourFiveFA30C['egenadbtc_W*m-3'], line_dash='dashed', line_width=2, line_color='#731517', legend="45% Fly Ash, T_i = 30C")

show(datFig01)

#### _Comparison with experiments_

In Phase I two "midscale" experiments were conducted: in both, concrete was poured into an insulated 4' $\times$ 4' $\times$ 6' = 1.22m $\times$ 1.22m $\times$ 1.83m ($x \times y \times z$ or width $\times$ depth $\times$ height) form.  The first, in January 2017, was of a baseline (no fly ash) mix without active cooling.  The second, in December 2018,  used a 25% fly ash mix and active cooling using piped water in PEX tubing.  Details may be found in:

- first midscale experiment: third quarterly report (March 31, 2017) and final report
- second midscale experiment: final report

Given the dimensions of these pours, the insulation on the sides and bottom, and the uninsulated top, these experiments can be approximated as 1-D.  The 1-D heat diffusion equation with the Arrhenius/Schindler $\dot e_{gen}$ model is then

$$ \rho c_v { {\partial T} \over {\partial t}} = k { {\partial^2 T} \over {\partial z^2}} + \dot e_{gen} $$

The boundary conditions were

- at the top surface, the heat flux is $\dot q = k \partial T / \partial z = h \cdot (T_s - T_{\infty})$ where $\dot q$ is heat flux, $h$ is the convection coefficient, $T_s$ is the temperature of the concrete's top surface, and $T_{\infty}$ is the ambient temperature (no solar radiation was present)
- at the bottom, an adiabatic surface was assumed: $\dot q = k \partial T / \partial z = 0$

The initial condition was $T(z,t=0) = T_i$.

A simple explicit finite-difference scheme (forward-time, centered-space) was coded in Python to solve this model.  Provisions were also made in this code for a relatively simple model of the cooling system and a "correction" to account for the heat loss through the formwork given the relatively long timescale of the experiment.

In what follows, solid lines represent temperatures somewhere in the "adiabatic core" of the concrete.  Dashed lines are at or near the top surface.  Black is experimental data, red is simulation data.

First, comparison of experimental temperature data with simulations for the first midscale experiment.

In [3]:
# first midscale experiment and simulation
PIexpVsimFME = pd.read_excel('PhaseI_ExperimentsVsSims.xlsx', 'FirstMidscaleExpJan2017', index_col=None, usecols=['t_exp_h', 'T_exp_.660mDegC', 'T_exp_surfaceDegC', 't_sim_h', 'T_sim_.662mDegC', 'T_sim_1.797mDegC'], na_values=['NA'])

# plot data
datFig02 = figure(#plot_width=1200, plot_height=700,
                  plot_width=900, plot_height=600,
                  tools=TOOLS,
                  title="First midscale experiment, baseline mix",
                  x_axis_label='Time, hours',
                  x_range=[0,175],
                  y_axis_label='Temperature, deg. C',
                  y_range=[10,70]
)

datFig02.line(PIexpVsimFME['t_exp_h'], PIexpVsimFME['T_exp_.660mDegC'], line_width=2, line_color='black', legend="Experimental data, z=0.66m")
datFig02.line(PIexpVsimFME['t_sim_h'], PIexpVsimFME['T_sim_.662mDegC'], line_width=2, line_color='#A8383B', legend="Simulation, z=0.66m")

datFig02.line(PIexpVsimFME['t_exp_h'], PIexpVsimFME['T_exp_surfaceDegC'], line_dash='dashed', line_width=1.5, line_color='black', legend="Experimental data, surface: z=1.83m")
datFig02.line(PIexpVsimFME['t_sim_h'], PIexpVsimFME['T_sim_1.797mDegC'], line_dash='dashed', line_width=1.5, line_color='#A8383B', legend="Simulation, z=1.797m")

datFig02.legend.location = "bottom_right"

show(datFig02)

Next, the second, actively cooled midscale experiment.  Simulated data have been shifted in time to account for time between batching and pouring of the actual concrete (note I haven't done this for the first midscale experiment above).  The temperatures plotted below are for a location in between cooling tubes:

In [4]:
# second midscale experiment and simulation
PIexpVsimSME = pd.read_excel('PhaseI_ExperimentsVsSims.xlsx', 'SecondMidscaleExpDec2018', index_col=None, usecols=['t_exp_h', 'T_exp_DegC', 't_sim_h', 'T_sim_DegC'], na_values=['NA'])

# plot data
datFig03 = figure(#plot_width=1200, plot_height=700,
                  plot_width=900, plot_height=600,
                  tools=TOOLS,
                  title="Second midscale experiment; 25% fly ash mix with active cooling",
                  x_axis_label='Time, hours',
                  x_range=[0,175],
                  y_axis_label='Temperature, deg. C',
                  y_range=[20,60]
)

datFig03.line(PIexpVsimSME['t_exp_h'], PIexpVsimSME['T_exp_DegC'], line_width=2, line_color='black', legend="Experimental data, sensor TS2")
datFig03.line(PIexpVsimSME['t_sim_h'], PIexpVsimSME['T_sim_DegC'], line_width=2, line_color='#A8383B', legend="Simulation, z=0.392m")

datFig03.legend.location = "bottom_right"

show(datFig03)

#### _Commentary_

In both cases, peak temperatures are well-modeled, although the time history of the temperature rise (and fall) are not.  Active cooling is reasonably modeled, at least far from the tubes.  Some conjectures, which are not necessarily mutually exclusive:

- In later times, simulated heat loss through the formwork (and perhaps top surface) is underestimated.
- Early rate of temperature rise underestimated - perhaps due to the second "hump" in a real $\dot e_{gen}$ curve that is not captured in the Arrhenius/Schindler model?  Assuming the total heat of hydration is accurately modeled, this would effectively shift more energy release to earlier times.  Eyeballing it, there may be an acceleration of temperature rise in the experimental data around hour 5 or so.  Further analysis, e.g. computing time derivatives from experimental and simulated data, would shed light on this, although computing derivatives can be noisy.  This may also explain the slower temperature fall later in simulations (simulations would have more residual exothermic reactions in later times).

It should be emphasized that the Schindler/Riding regressions were not evaluated, only used under the assumption that they are sufficiently accurate.